### 기본 세팅

In [1]:
# GPU 성능을 높이기 위한 라이브러리
!pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.45.2 # 처음 버전: 0.43.1
# !pip install --upgrade transformers bitsandbytes accelerate
# !pip install peft==0.10.0
!pip install fsspec==2024.12.0 gcsfs==2024.12.0
!pip install --upgrade datasets
!pip install -U transformers==4.41.0 trl==0.8.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, # 사전 학습된 모델과 토크나이저를 불러오기 위한 클래스
    TrainingArguments, # 트레이닝 과정의 다양한 파라미터 세팅
    BitsAndBytesConfig, # 모델의 메모리 최적화 관련 설정
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer

- 한국어 데이터셋을 훈련받은 LLama 모델 사용

In [3]:
base_model_path = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"

In [4]:
base_model_path

'beomi/Llama-3-Open-Ko-8B-Instruct-preview'

In [5]:
# GPU device_capablity가 8 이상이면 고급기능 사용.
if torch.cuda.get_device_capability()[0] >= 8:
    # !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

- QLoRA(양자화된 모델 로드하도록 설정)

In [6]:
# 메모리와 계산 속도 최적화
# 4비트 양자화된 모델
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

- 모델 로드

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=quant_config,
    # device_map="auto"
    device_map={"":0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

- 토크나이저 설정

In [8]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_path,
    trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # 문장 벡터화할 때 길이 맞춤

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


- PEFT (LoRA)

In [9]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

- 학습 파라미터 설정

In [10]:
training_params = TrainingArguments(
    output_dir="/content/drive/MyDrive/healthcare project/results",
    num_train_epochs=1, # 총 학습 epoch 수(데이터셋을 몇 번 반복할지)
    per_device_train_batch_size=4, # 배치 크기(한번에 처리하는 데이터 개수)
    # 배치 크기만큼의 데이터를 4번 처리한 후 가중치 업데이트.
    # 배치 크기를 키우지 않아 메모리는 절약하지만 학습효과는 [batch_size= 설정 배치크기 * step] 처럼 동작
    gradient_accumulation_steps=1, # epochs랑 동시에 사용 못하므로 epochs를 1이상으로 했으면 이건 1로
    gradient_checkpointing=True, # 메모리 절약용
    optim="paged_adamw_32bit", # 옵티마이저 설정
    save_steps=300,
    save_total_limit=1,  # 최근 1개 체크포인트만 유지(최근 1개만 저장)
    logging_steps=50,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False, # 원래 False 였음
    bf16=False,
    max_grad_norm=0.3, # 0.3 이상 변화 방지, 폭발적 그라디언트 방지
    max_steps=-1, # -1: 데이터셋 끝날 때까지 학습. 5000: 스텝 후 학습 종료
    warmup_ratio=0.03, # 전체 학습의 처음 3%동안 학습률 점진적으로 증가. 학습 초반에 서서히 학습
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

### 데이터 불러오기 & 학습 및 평가 데이터 나누기

In [11]:
# 데이터셋 로드
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/healthcare project/headache_qa_matching.csv', index_col=0)
df

,disease_category,disease_name,intention,question,final_answer
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?,만약 두통이 오랫동안 지속된다면 의사의 진단을 받아보는 것이 좋습니다. 스트레스와 ...
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?,긴장성 두통은 일상 생활에서 자세나 습관에 따라 발생할 수 있습니다. 특히 수면 자...
...,...,...,...,...,...
3423,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...,두통은 일상 생활에서 자주 경험할 수 있는 증상입니다. 두통은 다양한 원인에 의해 ...
3424,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...
3425,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.,두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법...
3426,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?,두통의 치료에는 다양한 방법이 있습니다. 주로 약물 요법이나 식이요법을 통해 증상을...


In [12]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.1,random_state=42)

In [13]:
len(train_df), len(val_df)

(3085, 343)

### 프롬프트 템플릿 생성
- instruct 모델이 경우 이 형식을 따름

In [14]:
def generate_prompt(example_df):
    prompt_list = []
    for i in range(len(example_df)):
        system_msg = "너는 의료 전문가 챗봇이야. 질문에 최대한 정확하고 자연스러운 대화체로 답변해줘."
        user_msg = example_df['question'].iloc[i]
        assistant_msg = example_df['final_answer'].iloc[i]

        text = (
            f"<|begin_of_text|>"
            f"<|system|>{system_msg}<|eot_id|>"
            f"<|user|>{user_msg}<|eot_id|>"
            f"<|assistant|>{assistant_msg}<|eot_id|>"
        )

        prompt_list.append({"text": text})

    return prompt_list

In [15]:
train_prompts = generate_prompt(train_df)
val_prompts = generate_prompt(val_df)

In [16]:
train_prompts[:2]

[{'text': '<|begin_of_text|><|system|>너는 의료 전문가 챗봇이야. 질문에 최대한 정확하고 자연스러운 대화체로 답변해줘.<|eot_id|><|user|>두통 진단을 받기 위해 어떤 의료진과 상담해야 하나요?<|eot_id|><|assistant|>두통은 많은 사람들이 경험하는 흔한 증상으로, 진단을 위해서는 정확한 병력 조사와 의료 기록 검토가 필요합니다. 두통은 다양한 원인에 의해 발생할 수 있으며, 일반적으로 의료 기관을 찾는 것이 가장 효과적인 방법입니다. 두통을 유발하는 기저 원인을 찾는 것이 중요하며, 이에 따라 적절한 진단 과정이 시작될 수 있습니다. 진단 과정에서는 여러 가지 검사를 통해 의사의 도움을 받을 수 있습니다. 두통은 주로 치료가 필요한 상황이 아니라 진단을 위해 의사와의 상담이 필요한 경우가 많습니다. 정확한 진단을 위해 의료 전문가의 도움을 받는 것이 중요합니다.<|eot_id|>'},
 {'text': '<|begin_of_text|><|system|>너는 의료 전문가 챗봇이야. 질문에 최대한 정확하고 자연스러운 대화체로 답변해줘.<|eot_id|><|user|>두통이 자주 발생하는데, 이는 뇌종양과 관련이 있을까요?<|eot_id|><|assistant|>두통은 일반적으로 머리와 관련된 불편한 증상으로 분류됩니다. 특히 편두통, 긴장성 두통, 군발성 두통 등이 있습니다. 이러한 두통은 뇌와 관련된 특정 부위에 영향을 미치는 경우가 많습니다. 뇌의 혈관, 신경 체계, 호르몬 분비 등의 변화로 인해 발생할 수 있습니다. 스트레스, 피로, 수면 부족, 환경 변화 등은 두통을 유발하는 주요 요인 중 하나입니다. 따라서, 두통은 다양한 원인에 의해 발생하며, 그 중에서도 뇌와 관련된 질환이 많습니다. 두통이 지속된다면 원인을 파악하여 적절한 치료와 예방을 받아야 합니다.<|eot_id|>'}]

In [ ]:
# dataset = []
# for idx, row in train_df.iterrows():
#     # print(idx, q['question'])
#     dataset.append({
#         'text': f"<s>[INST] {row['question']} [/INST]{row['final_answer']}</s>"
#     })

In [ ]:
# dataset[:2]

- 데이터셋 형식으로 변환하기

In [17]:
from datasets import Dataset

train_dataset = Dataset.from_list(train_prompts)
val_dataset = Dataset.from_list(val_prompts)

In [ ]:
# from datasets import Dataset

# dataset = Dataset.from_list(dataset)

In [18]:
len(train_dataset), train_dataset[:2]

(3085,
 {'text': ['<|begin_of_text|><|system|>너는 의료 전문가 챗봇이야. 질문에 최대한 정확하고 자연스러운 대화체로 답변해줘.<|eot_id|><|user|>두통 진단을 받기 위해 어떤 의료진과 상담해야 하나요?<|eot_id|><|assistant|>두통은 많은 사람들이 경험하는 흔한 증상으로, 진단을 위해서는 정확한 병력 조사와 의료 기록 검토가 필요합니다. 두통은 다양한 원인에 의해 발생할 수 있으며, 일반적으로 의료 기관을 찾는 것이 가장 효과적인 방법입니다. 두통을 유발하는 기저 원인을 찾는 것이 중요하며, 이에 따라 적절한 진단 과정이 시작될 수 있습니다. 진단 과정에서는 여러 가지 검사를 통해 의사의 도움을 받을 수 있습니다. 두통은 주로 치료가 필요한 상황이 아니라 진단을 위해 의사와의 상담이 필요한 경우가 많습니다. 정확한 진단을 위해 의료 전문가의 도움을 받는 것이 중요합니다.<|eot_id|>',
   '<|begin_of_text|><|system|>너는 의료 전문가 챗봇이야. 질문에 최대한 정확하고 자연스러운 대화체로 답변해줘.<|eot_id|><|user|>두통이 자주 발생하는데, 이는 뇌종양과 관련이 있을까요?<|eot_id|><|assistant|>두통은 일반적으로 머리와 관련된 불편한 증상으로 분류됩니다. 특히 편두통, 긴장성 두통, 군발성 두통 등이 있습니다. 이러한 두통은 뇌와 관련된 특정 부위에 영향을 미치는 경우가 많습니다. 뇌의 혈관, 신경 체계, 호르몬 분비 등의 변화로 인해 발생할 수 있습니다. 스트레스, 피로, 수면 부족, 환경 변화 등은 두통을 유발하는 주요 요인 중 하나입니다. 따라서, 두통은 다양한 원인에 의해 발생하며, 그 중에서도 뇌와 관련된 질환이 많습니다. 두통이 지속된다면 원인을 파악하여 적절한 치료와 예방을 받아야 합니다.<|eot_id|>']})

In [19]:
# SFTTrainer 클래스를 사용해 파인튜닝
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None, # 입력 시퀀스의 최대 길이를 별도로 제한하지 않음.
    tokenizer=tokenizer,
    args=training_params,
    packing=False
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3085 [00:00<?, ? examples/s]

Map:   0%|          | 0/343 [00:00<?, ? examples/s]

In [20]:
# 학습시기키
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,1.289600
100,1.061800
150,1.007500
200,0.944700
250,0.874000
300,0.824300
350,0.799000
400,0.762000
450,0.781100
500,0.711900


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `

TrainOutput(global_step=772, training_loss=0.819604186814066, metrics={'train_runtime': 5084.2847, 'train_samples_per_second': 0.607, 'train_steps_per_second': 0.152, 'total_flos': 3.710414386468454e+16, 'train_loss': 0.819604186814066, 'epoch': 1.0})

- 학습된 데이터 저장하기

In [21]:
trained_directory = "/content/drive/MyDrive/saved_trained_model_instruct"

# LoRA(PEFT) 모델 저장
trainer.model.save_pretrained(trained_directory)
tokenizer.save_pretrained(trained_directory)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/MyDrive/saved_trained_model_instruct/tokenizer_config.json',
 '/content/drive/MyDrive/saved_trained_model_instruct/special_tokens_map.json',
 '/content/drive/MyDrive/saved_trained_model_instruct/tokenizer.json')

In [22]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1.92


- 질문하기

In [23]:
def chat_response(model,tokenizer, user_input):

    messages = [
        {"role": "system", "content": "너는 의료 전문가 챗봇이야. 질문에 최대한 정확하고 자연스러운 대화체로 답변해줘."},
        {"role": "user", "content": user_input},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 생성 종료 조건을 정해줌.
    terminators = [
        tokenizer.eos_token_id, # 이 토큰을 만나면 생성 종료
        tokenizer.convert_tokens_to_ids("<|eot_id|>") # <|eot_id|> 이 토큰 만나도 생성 종료
    ]

    outputs = model.generate(
        input_ids,
        attention_mask=(input_ids != tokenizer.pad_token_id), # 패딩이 아닌 실제 텍스트만 골라서 마스크 생성(마스크: 필요한 정보만 놔두고 나머지 가림)
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.7,
        top_p=0.8,
    )
    response = outputs[0][input_ids.shape[-1]:]
    response_text = tokenizer.decode(response, skip_special_tokens=True)

    return response_text

In [24]:
chat_response(model, tokenizer, "오른쪽 머리에서 찌릿하는 두통은 뭔가요?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'두통은 다양한 이유로 발생할 수 있습니다. 두통은 일상적인 활동이나 스트레스와 관련된 요인으로 인해 증상이 나타나는 경향이 있습니다. 예를 들어, 두통은 피로, 우울감, 불안과 같은 감정 변화와 연관될 수 있습니다. 두통은 매우 흔한 증상이며, 다양한 증상과 함께 나타날 수 있습니다. 만약 두통이 지속된다면 의료 전문가와 상담하여 원인을 파악하고 적절한 치료 방법을 찾는 것이 중요합니다.'

In [25]:
chat_response(model, tokenizer, "두통의 증상이 심해질 경우 어떤 치료 방법이 효과적일까요?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'두통은 다양한 원인에 의해 발생할 수 있으며, 그 중 일부는 치료가 필요한 경우도 있습니다. 두통을 완화하기 위한 다양한 치료법이 있으며, 각 개인의 상황에 따라 가장 효과적인 방법을 선택할 수 있습니다. 주로 사용되는 치료법 중 하나는 진통제를 사용하는 것입니다. 진통제는 통증을 완화시키는 데 도움이 될 수 있으며, 때로는 치료를 위해 의사의 처방이 필요할 수도 있습니다. 다른 치료법으로는 편두통을 완화하기 위한 신경안정제를 사용하는 것이 있습니다. 신경안정제는 뇌의 혈액 공급을 감소시켜 통증을 완화시키는 데 도움이 됩니다. 또한, 약물 치료를 통해 두통을 완화할 수 있는 경우도 있지만, 때로는 심리적 치료도 효과적일 수 있습니다. 심리적 치료는 개인의 생각과 감정을 변화시키는 것을 목표로 합니다. 이를 통해 두통을 완화시키고 일상 생활에 복귀할 수 있도록 도와줍니다. 따라서, 두통이 심해진다면 의사와 상담하여 가장 효과적인 치료 방법을 찾아야 합니다.'

In [26]:
chat_response(model, tokenizer, "두통이라는 질병에서 머리 통증 이외에 어떤 증상들이 더 일반적으로 나타나나요?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'두통은 다양한 원인으로 인해 발생할 수 있으며, 이에는 두통 이외에도 다른 증상들이 동반될 수 있습니다. 뇌졸중으로 인한 두통은 일반적으로 두통과 함께 식증, 어지러움, 경련 등의 증상이 동반됩니다. 뇌종양으로 인한 두통은 일반적으로 두통과 함께 시야가 어지러울 수 있고, 경련 등의 증상이 동반될 수 있습니다. 위와 같은 증상들이 발생한다면, 의료 전문가와 상담하여 원인을 파악하고 적절한 치료를 받는 것이 중요합니다.'

- 저장한 가중치와 기본모델을 불러와 모델 다시 생성(가중치와 기본 모델을 병합해서 저장하지 않음)

In [ ]:
# 기본 모델 저장
# PEFT(LoRA) 적용된 모델이라면 기본 모델의 가중치는 저장되지 않으므로 기본 모델도 저장해야함.
# LoRA는 기본 모델의 가중치를 직접 변경하지 않고, 추가적인 적은 수의 가중치만 학습하는 방식임.
# 그래서 save_pretrained()를 호출하면 LoRA의 추가 가중치만 저장됨.
# trainer.model.base_model.save_pretrained(f"{save_directory}/base_model")

In [27]:
import os
import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, # 사전 학습된 모델과 토크나이저를 불러오기 위한 클래스
    TrainingArguments, # 트레이닝 과정의 다양한 파라미터 세팅
    BitsAndBytesConfig, # 모델의 메모리 최적화 관련 설정
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 학습때 설정한 양자화 옵션을 불러올 때도 그대로 넣어줘야함.
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# 기본 모델 로드
base_model_path = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map={"":0},
    quantization_config = quant_config,  # 양자화 적용
    torch_dtype=torch.float16  # 양자화된 모델을 불러올 때 dtype 설정
    )

# LoRA 가중치 적용
# 불러온 기본 모델(base_model)에 LoRA 가중치를 적용
trained_directory = "/content/drive/MyDrive/saved_trained_model_instruct"
merged_model = PeftModel.from_pretrained(base_model, trained_directory)

# 토크나이저 불러오기
merged_tokenizer = AutoTokenizer.from_pretrained(trained_directory)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


- 기본 모델과 가중치를 불러와 병합해서 저장

In [ ]:
# !pip install --upgrade transformers bitsandbytes accelerate

In [29]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

save_directory = "/content/drive/MyDrive/full_trained_model_instruct"

# LoRA 병합 (이제 LoRA 없이도 사용할 수 있음)
merged_model_for_save = merged_model.merge_and_unload()

# 전체 모델(기본 모델 + 학습한 가중치) 저장
merged_model_for_save.save_pretrained(save_directory)
merged_tokenizer.save_pretrained(save_directory)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('/content/drive/MyDrive/full_trained_model_instruct/tokenizer_config.json',
 '/content/drive/MyDrive/full_trained_model_instruct/special_tokens_map.json',
 '/content/drive/MyDrive/full_trained_model_instruct/tokenizer.json')

In [30]:
# 저장했던 모델 다시 로드
model_load = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/full_trained_model", device_map="auto")
tokenizer_load = AutoTokenizer.from_pretrained("/content/drive/MyDrive/full_trained_model")

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [31]:
import os
import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, # 사전 학습된 모델과 토크나이저를 불러오기 위한 클래스
    TrainingArguments, # 트레이닝 과정의 다양한 파라미터 세팅
    BitsAndBytesConfig, # 모델의 메모리 최적화 관련 설정
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer

In [32]:
chat_response(model, tokenizer, "두통의 증상이 심해질 경우 어떤 치료 방법이 효과적일까요?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'두통은 흔한 증상이며 많은 치료법이 있습니다. 두통은 치료하기 위해 여러 가지 방법을 시도할 수 있습니다. 일차성 두통의 경우에는 일상 생활에서 스트레스를 완화하고 휴식을 취하는 것이 도움이 될 수 있습니다. 또한 두통을 경감하기 위해 약물 치료도 사용될 수 있습니다. 약물 치료는 전문의의 처방과 지도 아래 진행되어야 합니다. 두통의 치료는 원인을 파악하고 적절한 치료를 받아야 합니다. 이를 위해 전문 의료 기관을 찾는 것이 가장 좋습니다.'

In [34]:
chat_response(model, tokenizer, "두통의 예방을 위해 어떤 자세나 활동이 효과적일까요?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'두통은 다양한 예방 방법이 있습니다. 먼저, 일상적인 생활 습관을 개선하는 것이 중요합니다. 스트레스를 효과적으로 관리하고, 충분한 수면과 휴식을 취하는 것이 필요합니다. 또한, 건강한 식습관을 유지하고, 신체 활동을 통해 스트레스를 완화할 수 있습니다. 마지막으로, 두통을 유발할 수 있는 음식을 피하는 것이 중요합니다. 따라서, 일상 생활에서 두통을 예방하기 위해 위의 방법들을 적용할 수 있습니다.'

In [ ]:
# query = '머리 전체가 아픈 원인이 뭐야?'
# pipe = pipeline(task='text-generation',
#                 model=model_load,
#                 tokenizer=tokenizer_load,
#                 max_length = 256)
# result = pipe(f"<s>[INST] {query} [/INST]")
# answer_result = result[0]['generated_text']
# answer_result